In [2]:
import mysql.connector
import pandas as pd
import statistics

In [32]:
# Connect to mysql database
cnx = mysql.connector.connect(user='root', password='Kh18riku!',
                                host='127.0.0.1',
                                database='global_n2o')
cursor = cnx.cursor()

query = ("SELECT * FROM RawMeasurementTreatment")
# Execute the query and fetch all results
alldata = pd.read_sql(query, cnx)
cnx.close()
         
print(alldata.head())

/home/ryan/.virtualenv/time/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


   RawID  TreatmentID        Date   N2OFlux  VWC  WFPS  SoilT     AirT  \
0      1            2  2006-06-19  -9.69777  NaN   NaN    NaN  24.2653   
1      2            2  2006-06-20  43.61360  NaN   NaN    NaN  20.4595   
2      3            2  2006-06-21  -4.27172  NaN   NaN    NaN  20.2586   
3      4            2  2006-06-22  53.78540  NaN   NaN    NaN  18.6777   
4      5            2  2006-06-23  23.54990  NaN   NaN    NaN  19.8857   

     Precip  NitrogenApplied  ... Management  DOY VWCcalculated  \
0  13.13860           0.6193  ...       None  170           NaN   
1   1.53634           0.0000  ...       None  171           NaN   
2   0.00000           0.0000  ...       None  172           NaN   
3   2.30792           0.4588  ...       None  173           NaN   
4   2.69813           4.7970  ...       None  174           NaN   

  WFPScalculated HMMCD           MCDscores  HMKNN  KNNscores  \
0            NaN     0  2.8303131064755176   None       None   
1            NaN     0  

In [27]:
# Connect to mysql database
cnx = mysql.connector.connect(user='root', password='Kh18riku!',
                                host='127.0.0.1',
                                database='global_n2o')
cursor = cnx.cursor()

query = ("SELECT * FROM ClassificationModel")
# Execute the query and fetch all results
olddata = pd.read_sql(query, cnx)
cnx.close()
         
print(olddata.head())

/home/ryan/.virtualenv/time/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


   LinMeasID  ExperimentID  PubID  RepID RepName  TreatmentID  IQRHM  HBOSHM  \
0          1             1      1      2    None            2    0.0       0   
1          2             1      1      2    None            2    0.0       0   
2          3             1      1      2    None            2    0.0       0   
3          4             1      1      2    None            2    0.0       0   
4          5             1      1      2    None            2    0.0       0   

   MADHM  HMTruth  ...  Sand  Silt  Clay  pH  SOM  PrimaryCrop    Management  \
0      0      NaN  ...  26.0  52.0  22.0 NaN  NaN      Alfalfa  Conventional   
1      0      NaN  ...  26.0  52.0  22.0 NaN  NaN      Alfalfa  Conventional   
2      0      NaN  ...  26.0  52.0  22.0 NaN  NaN      Alfalfa  Conventional   
3      0      NaN  ...  26.0  52.0  22.0 NaN  NaN      Alfalfa  Conventional   
4      0      NaN  ...  26.0  52.0  22.0 NaN  NaN      Alfalfa  Conventional   

   MAP  MAT  Latitude  
0  NaN  NaN   

In [33]:
fertdata = alldata[alldata['NitrogenApplied'] > 0]
dflen = len(fertdata.index)
i = 0

# For every row in alldata, set NitrogenForm and NitrogenType to the values in olddata where TreatmentID and Date match
for index, row in fertdata.iterrows():    
    TreatmentID = row['TreatmentID']
    Date = row['Date']
    # Convert the date to a datetime object
    Date = pd.to_datetime(Date)
    oldrow = olddata[(olddata['TreatmentID'] == TreatmentID) & (olddata['Date'] == Date)]
    if oldrow.empty:
        print("Empty")
        continue
    alldata.at[index, 'NitrogenForm'] = oldrow['NitrogenForm'].values[0]
    alldata.at[index, 'NitrogenType'] = oldrow['NitrogenType'].values[0]
    
    # Print progress 
    print("Percent Complete: ", i/dflen*100)
    i += 1

# Save the data to a csv file
alldata.to_csv('fert_treatment_data.csv')

/home/ryan/.virtualenv/time/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)


Percent Complete:  0.0
Percent Complete:  0.1953125
Percent Complete:  0.390625
Percent Complete:  0.5859375
Percent Complete:  0.78125
Percent Complete:  0.9765625
Percent Complete:  1.171875
Percent Complete:  1.3671875
Percent Complete:  1.5625
Percent Complete:  1.7578125
Percent Complete:  1.953125
Percent Complete:  2.1484375
Percent Complete:  2.34375
Percent Complete:  2.5390625
Percent Complete:  2.734375
Percent Complete:  2.9296875
Percent Complete:  3.125
Percent Complete:  3.3203125
Percent Complete:  3.515625
Percent Complete:  3.7109375
Percent Complete:  3.90625
Percent Complete:  4.1015625
Percent Complete:  4.296875
Percent Complete:  4.4921875
Percent Complete:  4.6875
Percent Complete:  4.8828125
Percent Complete:  5.078125
Percent Complete:  5.2734375
Empty
Percent Complete:  5.46875
Empty
Percent Complete:  5.6640625
Percent Complete:  5.859375
Percent Complete:  6.0546875
Empty
Percent Complete:  6.25
Empty
Percent Complete:  6.4453125
Percent Complete:  6.640625

In [34]:
# Read fert_treatment_data.csv
alldata = pd.read_csv('fert_treatment_data.csv')
alldata['FertList'] = None

# # For every row in alldata where NitrogenApplied is non 0, get all the values of NitrogenApplied from olddata where TreatmentID and Date match
fertdata = alldata[alldata['NitrogenApplied'] > 0]
dblen = len(alldata.index)
for index, row in fertdata.iterrows():
    TreatmentID = row['TreatmentID']
    fertdate = row['Date']
    # Convert the date to a datetime object
    fertdate = pd.to_datetime(fertdate)

    olddf = olddata[(olddata['TreatmentID'] == TreatmentID) & (olddata['Date'] == fertdate)]
    # If the olddf is empty, skip
    if olddf.empty:
        continue
    # Get a list of the values of NitrogenApplied
    nitrogen_list = olddf['NitrogenApplied'].values
    print(nitrogen_list)
    # Get the mode
    mode = statistics.mode(nitrogen_list)
    print(mode)
    # Set the value of NitrogenApplied to the mode
    alldata.at[index, 'NitrogenApplied'] = mode
    # Set the value of FertList to the list of values
    alldata.at[index, 'FertList'] = nitrogen_list
    # Print progress
    print("Percent Complete: ", index/dblen*100)

# Save the data to a csv file
alldata.to_csv('fert_treatment_data2.csv')
    

/home/ryan/.virtualenv/time/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)


[0.6193 0.6193]
0.6193
Percent Complete:  0.0
[0.4588 0.4588]
0.4588
Percent Complete:  0.001451456052329829
[4.797 4.797]
4.797
Percent Complete:  0.0019352747364397719
[7.5689 7.5689]
7.5689
Percent Complete:  0.003386730788769601
[11.5399 11.5399]
11.5399
Percent Complete:  0.0038705494728795437
[10.1282 10.1282]
10.1282
Percent Complete:  0.005322005525209373
[9.606 9.606]
9.606
Percent Complete:  0.006773461577539202
[10.4938 10.4938]
10.4938
Percent Complete:  0.00822491762986903
[39. 39.]
39.0
Percent Complete:  0.012579285786858517
[79. 79.]
79.0
Percent Complete:  0.029029121046596575
[110.]
110.0
Percent Complete:  0.3164174194079027
[120.]
120.0
Percent Complete:  0.3643154691347871
[87.]
87.0
Percent Complete:  0.5109125304200998
[101.]
101.0
Percent Complete:  0.6763785203857003
[168.]
168.0
Percent Complete:  0.7431454987928724
[120.]
120.0
Percent Complete:  0.7668526143142596
[120.]
120.0
Percent Complete:  0.8655516258726879
[120.]
120.0
Percent Complete:  0.9739270111

In [35]:
# Load the data from the csv file
alldata = pd.read_csv('fert_treatment_data2.csv')
# Drop rows where NitrogenApplied is 0
alldata = alldata[alldata['NitrogenApplied'] != 0]
# Drop rows where Nitrogen is nan
alldata = alldata.dropna(subset=['NitrogenApplied'])
# Save the data to a csv file
alldata.to_csv('fert_treatment_data3.csv')

In [44]:
# Load the data from the csv file
alldata = pd.read_csv('fert_treatment_data2.csv')

# If there is a value in NitrogenForm, drop the row
alldata2 = alldata.dropna(subset=['NitrogenType'])
# Get the inverse of alldata2
alldata = alldata[~alldata.index.isin(alldata2.index)]

# Print the unique values of NitrogenForm
print(alldata['NitrogenForm'].unique())



[nan '28-12-12' '10-34-0' 'Liquid 9 18 9' 'Starter Fertilizer' '7/27/2027'
 '10/20/2020' '10/30/2010' '19-17-0']


In [43]:
# Load the data from the csv file
alldata = pd.read_csv('fert_treatment_data2.csv')

# If NitrogenForm contains 'UAN', set NitrogenType to 'UAN'
alldata.loc[alldata['NitrogenForm'].str.contains('UAN', na=False), 'NitrogenType'] = 'UAN'

# If NitrogenForm contains 'Ammonium Nitrate', set NitrogenType to 'Ammonium Nitrate'
alldata.loc[alldata['NitrogenForm'].str.contains('Ammonium Nitrate', na=False), 'NitrogenType'] = 'Ammonium Nitrate'
# If NitrogenForm contains 'Ammonium Sulfate', set NitrogenType to 'Ammonium Sulfate'
alldata.loc[alldata['NitrogenForm'].str.contains('Ammonium Sulfate', na=False), 'NitrogenType'] = 'Ammonium Sulfate'
# If NitrogenForm contains 'Anhydrous', set NitrogenType to 'Anhydrous Ammonia'
alldata.loc[alldata['NitrogenForm'].str.contains('Anhydrous', na=False), 'NitrogenType'] = 'Anhydrous Ammonia'
# If NitrogenForm contains 'Urea', set NitrogenType to 'Urea'
alldata.loc[alldata['NitrogenForm'].str.contains('Urea', na=False), 'NitrogenType'] = 'Urea'
# If NitrogenForm contains 'Manure', set NitrogenType to 'Manure'
alldata.loc[alldata['NitrogenForm'].str.contains('Manure', na=False), 'NitrogenType'] = 'Manure'
# If NitrogenForm contains 'Litter', set NitrogenType to 'Manure'
alldata.loc[alldata['NitrogenForm'].str.contains('Litter', na=False), 'NitrogenType'] = 'Manure'
# If NitrogenForm contains 'ESN' or 'Agrotain' set NitrogenType to 'ESN'
alldata.loc[alldata['NitrogenForm'].str.contains('ESN|Agrotain', na=False), 'NitrogenType'] = 'ESN'
# If NitrogenForm = UAN, set NitrogenType to 'UAN'
alldata.loc[alldata['NitrogenForm'] == 'UAN', 'NitrogenForm'] = 'UAN'
# If NitrogenForm contains 'SuperU', set NitrogenType to 'ESN'
alldata.loc[alldata['NitrogenForm'].str.contains('SuperU', na=False), 'NitrogenType'] = 'ESN'

# Save the data to a csv file
alldata.to_csv('fert_treatment_data2.csv')



In [45]:
# Connect to mysql database
cnx = mysql.connector.connect(user='root', password='Kh18riku!',
                                host='127.0.0.1',
                                database='global_n2o')
cursor = cnx.cursor()

# Load csv
alldata = pd.read_csv('fert_treatment_data2.csv')
# Drop rows where NitrogenType is nan
alldata = alldata.dropna(subset=['NitrogenType'])

# Iterate through the rows of alldata
for index, row in alldata.iterrows():
    NitrogenType = row['NitrogenType']
    NitrogenForm = row['NitrogenForm']
    TreatmentID = row['TreatmentID']
    RawID = row['RawID']
    Date = row['Date']
    # Convert the date to a datetime object
    Date = pd.to_datetime(Date)
    # Update the table
    update = ("UPDATE RawMeasurementTreatment SET NitrogenForm = %s WHERE RawID = %s")
    cursor.execute(update, [NitrogenForm, RawID])
    update = ("UPDATE RawMeasurementTreatment SET NitrogenType = %s WHERE RawID = %s")
    cursor.execute(update, [NitrogenType, RawID])
cnx.commit()

In [46]:
## Connect to mysql database
cnx = mysql.connector.connect(user='root', password='Kh18riku!',
                                host='127.0.0.1',
                                database='global_n2o')
cursor = cnx.cursor()
query = ("SELECT * FROM RawMeasurementTreatment")
# Execute the query and fetch all results
alldata = pd.read_sql(query, cnx)
cnx.close()

# Keep only rows where NitrogenApplied > 0 and NitrogenType is nan
alldata = alldata[(alldata['NitrogenApplied'] > 0) & (alldata['NitrogenType'].isnull())]

# If NitrogenApplied > 0 and NitrogenType is nan, set NitrogenType to 'Unknown'
alldata.loc[(alldata['NitrogenApplied'] > 0) & (alldata['NitrogenType'].isnull()), 'NitrogenType'] = 'Unknown'

# Connect to mysql database
cnx = mysql.connector.connect(user='root', password='Kh18riku!',
                                host='127.0.0.1',
                                database='global_n2o')
cursor = cnx.cursor()

# Iterate through the rows of alldata
for index, row in alldata.iterrows():
    NitrogenType = row['NitrogenType']
    RawID = row['RawID']
    # Update the table
    update = ("UPDATE RawMeasurementTreatment SET NitrogenType = %s WHERE RawID = %s")
    cursor.execute(update, [NitrogenType, RawID])
cnx.commit()

/home/ryan/.virtualenv/time/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [8]:
# Print 
print(alldata['VWC'].describe())

count    48157.000000
mean         0.267288
std          0.113499
min          0.000000
25%          0.187705
50%          0.263000
75%          0.339685
max          0.960000
Name: VWC, dtype: float64


In [13]:
# Connect to mysql database and update the table
cnx = mysql.connector.connect(user='root', password='Kh18riku!',
                                host='127.0.0.1',
                                database='global_n2o')
cursor = cnx.cursor()

for index, row in alldata.iterrows():
    vwc = row['VWC']
    # skip if vwc is nan
    if pd.isnull(vwc):
        continue
    RawID = row['RawID']
    update = ("UPDATE RawMeasurement SET VWC = %s WHERE RawID = %s")
    cursor.execute(update, [vwc, RawID])
cnx.commit()
cnx.close()


In [4]:
# Connect to mysql database
cnx = mysql.connector.connect(user='root', password='Kh18riku!',
                                host='127.0.0.1',
                                database='global_n2o')
cursor = cnx.cursor()

query = ("SELECT * FROM Treatment")
# Execute the query and fetch all results
treatmentdata = pd.read_sql(query, cnx)
cnx.close()
        

/home/ryan/.virtualenv/time/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [5]:
# Connect to mysql database
cnx = mysql.connector.connect(user='root', password='Kh18riku!',
                                host='127.0.0.1',
                                database='global_n2o')
cursor = cnx.cursor()

query = ("SELECT * FROM Replication")
# Execute the query and fetch all results
repdata = pd.read_sql(query, cnx)
cnx.close()
        

/home/ryan/.virtualenv/time/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [6]:
# for each row in treatmentdata, get the mean sand, silt, and clay values from repdata where TreatmentID matches
for index, row in treatmentdata.iterrows():
    TreatmentID = row['TreatmentID']
    repdata2 = repdata[repdata['TreatmentID'] == TreatmentID]
    sand = repdata2['Sand'].mean()
    silt = repdata2['Silt'].mean()
    clay = repdata2['Clay'].mean()
    treatmentdata.at[index, 'SandMean'] = sand
    treatmentdata.at[index, 'SiltMean'] = silt
    treatmentdata.at[index, 'ClayMean'] = clay
    print("Percent Complete: ", index/len(treatmentdata.index)*100)
    
treatmentdata.to_csv('treatment_data.csv')

Percent Complete:  0.0
Percent Complete:  0.3115264797507788
Percent Complete:  0.6230529595015576
Percent Complete:  0.9345794392523363
Percent Complete:  1.2461059190031152
Percent Complete:  1.557632398753894
Percent Complete:  1.8691588785046727
Percent Complete:  2.1806853582554515
Percent Complete:  2.4922118380062304
Percent Complete:  2.803738317757009
Percent Complete:  3.115264797507788
Percent Complete:  3.4267912772585665
Percent Complete:  3.7383177570093453
Percent Complete:  4.049844236760125
Percent Complete:  4.361370716510903
Percent Complete:  4.672897196261682
Percent Complete:  4.984423676012461
Percent Complete:  5.29595015576324
Percent Complete:  5.607476635514018
Percent Complete:  5.919003115264798
Percent Complete:  6.230529595015576
Percent Complete:  6.5420560747663545
Percent Complete:  6.853582554517133
Percent Complete:  7.165109034267912
Percent Complete:  7.476635514018691
Percent Complete:  7.78816199376947
Percent Complete:  8.09968847352025
Percent 

In [13]:
# Log in to the database
cnx = mysql.connector.connect(user='root', password='Kh18riku!',
                                host='127.0.0.1',
                                database='global_n2o')
cursor = cnx.cursor()

# Load the data from the csv file
treatmentdata = pd.read_csv('treatment_data.csv')

# Iterate through the rows of treatmentdata and update the Sand, Silt, and Clay columns
for index, row in treatmentdata.iterrows():
    print("Percent Complete: ", index/len(treatmentdata.index)*100)
    Sand = row['SandMean']
    Silt = row['SiltMean']
    Clay = row['ClayMean']
    # If any of the values are nan, skip
    if pd.isnull(Sand) or pd.isnull(Silt) or pd.isnull(Clay):
        print('empty data')
        continue
    TreatmentID = row['TreatmentID']
    update = ("UPDATE Treatment SET SandMean = %s WHERE TreatmentID = %s")
    cursor.execute(update, [Sand, TreatmentID])
    update = ("UPDATE Treatment SET SiltMean = %s WHERE TreatmentID = %s")
    cursor.execute(update, [Silt, TreatmentID])
    update = ("UPDATE Treatment SET ClayMean = %s WHERE TreatmentID = %s")
    cursor.execute(update, [Clay, TreatmentID])
cnx.commit()
cnx.close()

Percent Complete:  0.0
Percent Complete:  0.3115264797507788
Percent Complete:  0.6230529595015576
Percent Complete:  0.9345794392523363
empty data
Percent Complete:  1.2461059190031152
Percent Complete:  1.557632398753894
empty data
Percent Complete:  1.8691588785046727
empty data
Percent Complete:  2.1806853582554515
Percent Complete:  2.4922118380062304
Percent Complete:  2.803738317757009
Percent Complete:  3.115264797507788
Percent Complete:  3.4267912772585665
Percent Complete:  3.7383177570093453
Percent Complete:  4.049844236760125
Percent Complete:  4.361370716510903
Percent Complete:  4.672897196261682
Percent Complete:  4.984423676012461
Percent Complete:  5.29595015576324
Percent Complete:  5.607476635514018
Percent Complete:  5.919003115264798
Percent Complete:  6.230529595015576
Percent Complete:  6.5420560747663545
Percent Complete:  6.853582554517133
Percent Complete:  7.165109034267912
Percent Complete:  7.476635514018691
Percent Complete:  7.78816199376947
Percent Com

In [3]:
# Drop all rows where BulkDensity is nan
alldata = alldata.dropna(subset=['BulkDensity'])
# Drop all rows where both VWC and WFPS are nan at the same time
alldata = alldata.dropna(subset=['VWC', 'WFPS'], how='all')
novwc = alldata[alldata['VWC'].isnull()]
nowfps = alldata[alldata['WFPS'].isnull()]

#Describe the data
print(novwc['VWC'].describe())
print(nowfps['WFPS'].describe())

count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: VWC, dtype: float64
count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: WFPS, dtype: float64


In [7]:
print(novwc.head())
print(len(novwc.index))
print(len(nowfps.index))


      RawID  ExperimentID  PubID  RepID        Date  DOY  N2OFlux  VWC  \
904  646898             4      4      5  2012-01-02    2  7.11502  NaN   
906  646900             4      4      5  2012-01-04    4  6.37945  NaN   
909  646903             4      4      5  2012-01-07    7      NaN  NaN   
912  646906             4      4      5  2012-01-10   10  6.47797  NaN   
914  646908             4      4      5  2012-01-12   12  6.08204  NaN   

         WFPS    SoilT  ...  Silt  Clay  pH BulkDensity  SOM      PrimaryCrop  \
904  0.536811  9.83154  ...  33.7  54.3 NaN        1.45  NaN  Zea mays (Corn)   
906  0.542229  7.59185  ...  33.7  54.3 NaN        1.45  NaN  Zea mays (Corn)   
909  0.533342      NaN  ...  33.7  54.3 NaN        1.45  NaN  Zea mays (Corn)   
912  0.531475  4.19921  ...  33.7  54.3 NaN        1.45  NaN  Zea mays (Corn)   
914  0.515219  2.83692  ...  33.7  54.3 NaN        1.45  NaN  Zea mays (Corn)   

       Management  MAP  MAT  Latitude  
904  Conventional  NaN  NaN 

In [5]:
cnx = mysql.connector.connect(user='root', password='Kh18riku!',
                                host='127.0.0.1',
                                database='global_n2o')
cursor = cnx.cursor()

for index, row in novwc.iterrows():
    wfps = row['WFPS']
    bd = row['BulkDensity']
    vwc = wfps*(1 - bd / 2.65)
    RawID = row['RawID']
    # Update the table
    
    update = ("UPDATE RawMeasurement SET VWCcalculated = %s WHERE RawID = %s")
    cursor.execute(update, [vwc, RawID])
cnx.commit()
cnx.close()

In [8]:
cnx = mysql.connector.connect(user='root', password='Kh18riku!',
                                host='127.0.0.1',
                                database='global_n2o')
cursor = cnx.cursor()

for index, row in nowfps.iterrows():
    vwc = row['VWC']
    bd = row['BulkDensity']
    wfps = vwc / (1 - bd / 2.65)
    RawID = row['RawID']
    # Update the table
    
    update = ("UPDATE RawMeasurement SET WFPScalculated = %s WHERE RawID = %s")
    cursor.execute(update, [wfps, RawID])
cnx.commit()
cnx.close()